In [38]:
%matplotib inline
%pylab inline
from sklearn.model_selection import train_test_split , RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from matplotlib.colors import LogNorm
from sklearn.datasets import make_gaussian_quantiles
from sklearn.ensemble import AdaBoostClassifier , RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

Populating the interactive namespace from numpy and matplotlib


In [2]:
cd ..

C:\Users\ChiefGuti\Documents\Lund University\Thesis\MLImplementacion


In [3]:
cd QAM4bit/

C:\Users\ChiefGuti\Documents\Lund University\Thesis\MLImplementacion\QAM4bit


In [4]:
def BER(x,y):
    j=0
    for i in range(len(x)):
        if(x[i]!=y[i]):
            j+=1
    return(j/len(x))
def ColorMap(Word,Map,M):
    Cmap=arange(M)
    for i in range(len(Word)):
        for j in range(len(Cmap)):
            if(Word[i]==Org[j]):
                Map[i]=Cmap[j]
def NormData(DataSet,data):
    for i in range(len(data[:,0])):
        DataSet[i,0]=data[:,0][i]
        DataSet[i,1]=data[:,1][i]
    DataSet[:,0]=DataSet[:,0]/max(DataSet[:,0])
    DataSet[:,1]=DataSet[:,1]/max(DataSet[:,1])

In [5]:
Code=loadtxt('PBRSout.txt',skiprows=5)

In [6]:
INT=[]
for i in range(int(len(Code)/4)):
    INT.append(i*4)
CodeWord=[]
for i in range(len(Code)):
    CodeWord.append(str(int(Code[i])))

Word=[]
Org=[]
j=0
for i in INT:
    Word.append(CodeWord[i]+CodeWord[i+1]+CodeWord[i+2]+CodeWord[i+3])
    
    if((Word[j] in Org)==False ):
        Org.append(Word[j])
    j=j+1
Map=zeros(len(Word))
ColorMap(Word,Map,16)

In [7]:
R=[3,4,5,6,8]
OSNR=[15,17,19,20,22,24,25]

In [8]:
data = loadtxt('OutRx_DualPol4bOSNR15R2.txt',delimiter=';')
data=data[1::2][:]
DataSet=zeros((len(data[:,0]),2))
NormData(DataSet,data)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(DataSet, Map, test_size = 0.20)


In [28]:
dt = RandomForestClassifier(n_estimators=100, max_depth=2,random_state=0)
dt.fit(X_train, y_train)
 

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=2, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [41]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.grid_search import GridSearchCV

param_grid = {"base_estimator__criterion" : ["gini", "entropy"],
              "base_estimator__splitter" :   ["best", "random"],
              "n_estimators": [1, 2]
             }


DTC = DecisionTreeClassifier(random_state = 11, max_features = "auto", class_weight = "auto",max_depth = None)

ABC = AdaBoostClassifier(base_estimator = DTC)

# run grid search
grid_search_ABC = GridSearchCV(ABC, param_grid=param_grid, scoring = 'roc_auc')

C:\Users\ChiefGuti\Miniconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\ChiefGuti\Miniconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [43]:
DTC = DecisionTreeClassifier(random_state = 11, max_features = "auto", class_weight = "balanced",max_depth = None)
dt = AdaBoostClassifier(base_estimator=DTC,learning_rate=)
dt.fit(X_train, y_train)
 

AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=DecisionTreeClassifier(class_weight='balanced', criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=11,
            splitter='best'),
          learning_rate=1.0, n_estimators=50, random_state=None)

In [44]:
y_prediction=dt.predict(X_test)

In [45]:
print(confusion_matrix(y_test,y_prediction))
print(classification_report(y_test,y_prediction))

[[140   1   0   0   0 263   0   0   0   0   0   0   6   0   0   0]
 [  4 134   0   0   0   2   0   0   0   0   0   0 248   0   0   0]
 [  0   0 123   0   0   0   0   0   0   2   0   0   0   0   2 304]
 [  0   0   0  74   0   0   0   1   0   0   0   0 341   0   4   0]
 [  0   0   2   0   0   0   0   0   0 116   0   0   0   0 271   3]
 [  0   0   0   0   0 391   0   0   0   0   0   0   7   0   0   7]
 [  0   0 131   0   0   0  28   0   0   0   0  53   0   0   0 215]
 [  0   0   0   1   0   0   0  58   0   0   0   0   2   0 348   0]
 [111   0   0   0   0 235   0   0  15   0   0   0   0  43   0   0]
 [  0   0   0   0   0   0   0  54   0 135   0   0   0   0 216   0]
 [  0 119   0  38   0   0   0   0   0   0  23   0 239   0   0   0]
 [  0   0   0   0   0   8   0   0   0   0   0  80   0   3   0 311]
 [  0   0   0   0   0  16   0   0   0   0   0   0 386   0   7   0]
 [  0   0   0   0   0 307   0   0   0   0   0   2   0  82   0   4]
 [  0   0   0   0   0   0   0   0   0   0   0   0   4   0 388 

C:\Users\ChiefGuti\Miniconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [46]:
BER(y_test,y_prediction)

0.6234360695758315